# Week-13: Classification Task in Python

<font size='4'>

* Welcome back!
* Today, we will go over a classification example using `skicit-learn` package in Python.

<font size='4'>

* HW8 Q1: The location of files are misplaced. Suppose that `python_proj` is your working directory, <br> `HW8_main.py` should be put right under `python_proj`, while `HW8Fun.py` should be put under `self_py_fun` folder.
* Interpretation of parameters of the linear model is not correct:
<img src="figures/HW8_linear_model_output.png" alt="drawing" width="500"/>

* For continuous covariate, i.e., CTQ total score, you should write:
    * One score increase in CTQ total score, on average, corresponds to a 0.042 score increase in PCL5 score at intake, adjusting for other covariates (p=0.025, 95% CI: [0.003, 0.081]).
* For categorical covariate, i.e., military rank, you should have reference group:
    * Patients who are officers, on average, had lower PCL5 score at intake by 3.06 than patients who are enlisted, adjusting for other covariates (p=0.022, 95% CI: [0.44, 5.67]). 

## Import data

<font size='4'>

* `Scikit-learn`, known as `sklearn`, is an open-source, robust library for machine learning in Python.
* It is created to streamline the process of implementing machine learning and statistical models in Python.
* The package comes with standard machine learning datasets, and you can import it without downloading them from an external website or database.
* Since we will go over a classification example, we will be using the [`wine dataset`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine) (Click it for more information).

In [77]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report

In [78]:
wine_data = load_wine()

<font size='4'>

* Executing the above code returns a dictionary-like object (we just learned!) that contains data and metadata.
    * **Metadata**: This is a terminology for data dictionary, i.e., a description of the data itself.

In [79]:
# Convert the data to a Pandas dataframe
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Add the target label (add a new column)
wine_df['target'] = wine_data.target

# Preview
wine_df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


### Exploratory Data Analysis

<font size='4'>

* Before conducting any data analysis, always check the quality of the dataset with exploratory data analysis.
* You can call `.info()` method to print out a summary of each column.

In [80]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  targe

<font size='4'>

* There are 178 data samples with 14 columns including the target column (output that we would like to predict).
* Luckily, no missing values are in our dataset from `Non-Null Count`.
* All features are `float64` except for target column.
* The dataset consumes 19.6 KB of memory.

In [81]:
wine_df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


<font size='4'>

* Since `target` is a categorical variable, we check the frequency and proportion using `.value_counts()` method.

In [82]:
pd.concat([wine_df['target'].value_counts(), wine_df['target'].value_counts(normalize=True)*100], axis=1)

,count,proportion
target,,
1,71,39.887640
0,59,33.146067
2,48,26.966292


## Data preprocessing

<font size='4'>

* Preprocessing is important prior to applying machine learning algorithms.
    * Check missing values, outliers, duplicates, errors, and data types.
    * Avoid "Garbage in, garbage out."
* Machine learning models typically require numerical inputs.
* Another practice is to standardize the input (via Z-transform) to make predictors more comparable. 
    * We do not want predictor A has a magnitude of 10000, while predictor B has a magnitude of 0.1.
    * We can achieve the goal using `StandardScaler()` class.
    * Each value goes through the following transformation columnwise, i.e., $x = (x - x_{mean})/x_{sd}$.

In [83]:
# Remember to import standardscaler (code should be written in the beginning of the Jupyter notebook).
# Split data into features (input) and label (output)
print(wine_data.feature_names)

# Always make a copy to avoid making changes on the raw data (when you have sufficient amount of memory).
x_mat = wine_df[wine_data.feature_names].copy()
y_val = wine_df['target'].copy()

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


<font size='4'>
<img src="figures/sklearn_flowchart_1.png" alt="drawing" width="900"/>
    
* In this flowchart, no output data are involved. We use `.transform()` in the last step.

In [84]:
# Instantiate scaler and fit on features
scaler_obj = StandardScaler()

# apply changes to training data
# and update parameters (in this case, no model parameters are available, so this is optional)
scaler_obj.fit(x_mat)

# apply changes to any data and assign it with another variable
x_scaled_mat = scaler_obj.transform(x_mat)

# view the transformed output
print(type(x_scaled_mat))
print(x_scaled_mat.shape)
x_scaled_mat[0, :]

<class 'numpy.ndarray'>
(178, 13)


array([ 1.51861254, -0.5622498 ,  0.23205254, -1.16959318,  1.91390522,
        0.80899739,  1.03481896, -0.65956311,  1.22488398,  0.25171685,
        0.36217728,  1.84791957,  1.01300893])

## Model training

<font size='4'>

* As we previously mentioned, we need to have training and testing set when we perform classification tasks.
* If rows of input data are independent of each other, we can randomly select training and testing set.
* Sklearn package has a built-in function called `train_test_split()`.
* We usually set 70% data to train and 30% to test. The exact ratio vary depending on the volume of the data.

In [85]:
# remember to import train_test_split in the beginning
x_train_scaled, x_test_scaled, y_train, y_test = train_test_split(x_scaled_mat, y_val, train_size=0.7, random_state=1)

# Check the splits are correct
print(x_train_scaled.shape[0])
print(x_test_scaled.shape[0])

124
54


### Model building

<font size='4'>

* Sklearn has numerous built-in classification methods. We will demonstrate a couple of methods including
    * logistic regression
    * support vector machine
    * decision tree classifier
    
* <img src="figures/sklearn_flowchart_2.png" alt="drawing" width="900"/>

    * In this flowchart, both input and output (in the training set) are involved, so `.predict()` is used finally.

In [86]:
# Instnatiating the models 
# Sometimes, you need to modify the parameters inside each of the function.
# If not, the model will use its default values.
logistic_obj = LogisticRegression()
svm_obj = SVC(probability=True) # probability is set to be False by default.
tree_obj = DecisionTreeClassifier()

# Training the models 
logistic_obj.fit(x_train_scaled, y_train)
svm_obj.fit(x_train_scaled, y_train)
tree_obj.fit(x_train_scaled, y_train)

# Making predictions with each model
log_reg_preds = logistic_obj.predict(x_test_scaled)
svm_preds = svm_obj.predict(x_test_scaled)
tree_preds = tree_obj.predict(x_test_scaled)

In [87]:
y_preds_three_method = np.stack([log_reg_preds, svm_preds, tree_preds], axis=0).T
y_preds_three_method[:5, :]

array([[2, 2, 2],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0]])

In [88]:
# You can view the probability vector per measure per method.
log_reg_probs = logistic_obj.predict_proba(x_test_scaled)
svm_probs = svm_obj.predict_proba(x_test_scaled)
tree_probs = tree_obj.predict_proba(x_test_scaled)
print(log_reg_probs[:5,:])
print(svm_probs[:5,:])
print(tree_probs[:5,:])

[[2.63329272e-02 1.34031306e-02 9.60263942e-01]
 [4.69277794e-04 9.99390372e-01 1.40350652e-04]
 [9.97118302e-01 2.30543045e-03 5.76267446e-04]
 [8.50282835e-03 9.91447420e-01 4.97517215e-05]
 [9.99257734e-01 2.14428342e-04 5.27837453e-04]]
[[0.0108554  0.01409034 0.97505427]
 [0.00332249 0.99504603 0.00163148]
 [0.98987005 0.00319413 0.00693582]
 [0.03524411 0.93126552 0.03349038]
 [0.98826474 0.00329393 0.00844133]]
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


## Model Evaluation

<font size='4'>

* In our case, we will use `classification_report()` to build a text report showing main classification metrics such as `precision`, `recall`, `f1_score`, `accuracy`, etc.

In [89]:
# from sklearn.metrics import classification_report (put it in the beginning)
# Store model predictions in a dictionary
# this makes it easier to iterate through each model
# and print the results. 
model_preds = {
    "Logistic Regression": log_reg_preds,
    "Support Vector Machine": svm_preds,
    "Decision Tree": tree_preds
}

for model, preds in model_preds.items():
    print('{} Results:\n {}\n'.format(model, classification_report(y_test, preds, output_dict=True)))

Logistic Regression Results:
 {'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 23.0}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 19.0}, '2': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 12.0}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 54.0}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 54.0}}

Support Vector Machine Results:
 {'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 23.0}, '1': {'precision': 0.95, 'recall': 1.0, 'f1-score': 0.9743589743589743, 'support': 19.0}, '2': {'precision': 1.0, 'recall': 0.9166666666666666, 'f1-score': 0.9565217391304348, 'support': 12.0}, 'accuracy': 0.9814814814814815, 'macro avg': {'precision': 0.9833333333333334, 'recall': 0.9722222222222222, 'f1-score': 0.976960237829803, 'support': 54.0}, 'weighted avg': {'precision': 0.9824074074074074, 'recall': 0.9814814814814815, 'f1-score': 0.98131632

In [90]:
for model, preds in model_preds.items():
    print(f"{model} Results:\n{classification_report(y_test, preds)}", sep="\n\n") # output_dict=False by default

Logistic Regression Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        19
           2       1.00      1.00      1.00        12

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

Support Vector Machine Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       0.95      1.00      0.97        19
           2       1.00      0.92      0.96        12

    accuracy                           0.98        54
   macro avg       0.98      0.97      0.98        54
weighted avg       0.98      0.98      0.98        54

Decision Tree Results:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        23
           1       0.95      0.95      0.95  

<font size='4'>
The reported averages include 

* (sample) average (only for multilabel classification). 
* macro average (averaging the unweighted mean per label), 
* weighted average (averaging the support-weighted mean per label).

Which method performs the best?